In [1]:
import lxml.html as html
import pandas as pd
import pyspark
import requests
import sys
import time

In [2]:
start = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227"

In [3]:
response = requests.get(start)
response.status_code
page = html.document_fromstring(response.text)

In [4]:
e = page.getchildren().pop()
e

<Element body at 0x8e5a0e8>

In [5]:
regions = e.xpath('//nobr//a')
len(regions)

87

In [6]:
start_time = time.time()


list_of_correct_tables = []
r = 0
t = 0

for region in regions[:]:
    
    try:
        region_name = region.text
        region_url = region.attrib['href']
        print(r, region_name)
    
        response = requests.get(region_url)
        assert response.status_code == 200
    except:
        print("Unexpected error:", sys.exc_info()[0], region_name)
        continue
    
    region_page_body = html.document_fromstring(response.text).getchildren().pop()
    tiks =  region_page_body.xpath('//nobr//a')
    if len(tiks) == 0:
        print("0 tiks")
    
    for tik in tiks[:]:
        
        try:
            tik_name = tik.text
            tik_url = tik.attrib['href']
            
            print(t, tik_name)
            
            response = requests.get(tik_url)
            assert response.status_code == 200
            
            tik_page_body = html.document_fromstring(response.text).getchildren().pop()
            uiks_url = tik_page_body.xpath('//td//a[text()="сайт избирательной комиссии субъекта Российской Федерации"]/@href').pop()

            response = requests.get(uiks_url)
            assert response.status_code == 200
            
            uiks_page_body = html.document_fromstring(response.text).getchildren().pop()

            table_uiks = uiks_page_body.xpath('//td//div//table').pop()

            table = pd.read_html(html.tostring(table_uiks), header=0).pop()
            table.drop(12, inplace=True)

            region_row = [region_name for x in range(len(table.columns))]
            tik_row = [tik_name for x in range(len(table.columns))]
            df = pd.DataFrame([region_row, tik_row], columns=table.columns)
            df = df.append(table, ignore_index=True)
            list_of_correct_tables.append(df)
            t+=1
            
        except:
            print("Unexpected error:", sys.exc_info()[0], region_name, tik_name)
            t += 1
            continue
        
    r += 1
    
    
finish = start_time - time.time()


finish - start_time

0 Республика Адыгея (Адыгея)
0 Адыгейская
1 Гиагинская
2 Кошехабльская
3 Красногвардейская
4 Майкопская 
5 Майкопская городская
6 Тахтамукайская 
7 Теучежская
8 Шовгеновская
1 Республика Алтай
9 Горно-Алтайская городская
10 Кош-Агачская
11 Майминская
12 Онгудайская
13 Турочакская
14 Улаганская
15 Усть-Канская
16 Усть-Коксинская
17 Чемальская
18 Чойская
19 Шебалинская
2 Республика Башкортостан
20 Абзелиловская
21 Агидельская городская
22 Альшеевская
23 Архангельская
24 Аскинская
25 Аургазинская
26 Баймакская
27 Бакалинская
28 Балтачевская
29 Белебеевская
30 Белокатайская
31 Белорецкая
32 Бижбулякская
33 Благоварская
34 Благовещенская
35 Буздякская
36 Бураевская
37 Бурзянская
38 Гафурийская
39 Давлекановская
40 Дуванская
41 Дюртюлинская
42 Ермекеевская
43 Зианчуринская
44 Зилаирская
45 Иглинская
46 Илишевская
47 Ишимбайская
48 Калтасинская
49 Караидельская
50 Кармаскалинская
51 Кигинская
52 Краснокамская
53 Кугарчинская
54 Кумертауская городская
55 Кушнаренковская
56 Мечетлинская
57 Мишк

422 Нурлатская
423 Нурлатская городская
424 Пестречинская
425 Рыбно-Слободская
426 Сабинская
427 Сармановская
428 Спасская
429 Тетюшская
430 Тукаевская
431 Тюлячинская
432 Черемшанская
433 Чистопольская
434 Чистопольская городская
435 Ютазинская
17 Республика Тыва
436 Ак-Довуракская
437 Бай-Тайгинская
438 Барун-Хемчикская
439 Дзун-Хемчикская
440 Каа-Хемская
441 Кызылская городская
442 Кызылская
443 Монгун-Тайгинская
444 Овюрская
445 Пий-Хемская
446 Сут-Хольская
447 Тандинская
448 Тес-Хемская
449 Тоджинская
450 Улуг-Хемская
451 Чаа-Хольская
452 Чеди-Хольская
453 Эрзинская
454 Тере-Хольская
18 Удмуртская Республика
455 Алнашская
456 Балезинская
457 Вавожская
458 Воткинская
459 Воткинская городская
460 Глазовская
461 Глазовская городская
462 Граховская
463 Дебесская
464 Завьяловская
465 Игринская
466 Ижевск, Индустриальная
467 Ижевск, Ленинская
468 Ижевск, Октябрьская
469 Ижевск, Первомайская
470 Ижевск, Устиновская
471 Камбарская
472 Каракулинская
473 Кезская
474 Кизнерская
475 Киясовска

847 Чердынская
848 Чернушинская
849 Чусовская
850 Звездная городская
851 Кунгурская районная
852 Соликамская районная
853 Гайнская
854 Косинская
855 Кочевская
856 Кудымкарская районная
857 Кудымкарская городская
858 Юрлинская
859 Юсьвинская
860 Пермь, Свердловская № 1
28 Приморский край
861 Анучинская
862 Арсеньевская городская
863 Артёмовская городская
864 Большекаменская городская
865 Владивосток, Ленинская
866 Владивосток, Первомайская
867 Владивосток, Первореченская
868 Владивосток, Советская
869 Владивосток, Фрунзенская
870 Дальнегорская городская
871 Дальнереченская городская
872 Кавалеровская
873 Кировская
874 Красноармейская
875 Лазовская
876 Лесозаводская городская
877 Михайловская
878 Надеждинская
879 Находкинская городская
880 Октябрьская
881 Ольгинская
882 Партизанская городская
883 Партизанская
884 Пограничная
885 Пожарская
886 Спасская городская
887 Тернейская
888 Уссурийская городская
889 Ханкайская
890 Хасанская
891 Хорольская
892 Черниговская
893 Чугуевская
894 Фокинск

1242 Лухская
1243 Палехская
1244 Пестяковская
1245 Приволжская
1246 Пучежская
1247 Родниковская
1248 Савинская
1249 Тейковская
1250 Тейковская городская
1251 Фурмановская
1252 Шуйская
1253 Шуйская городская
1254 Южская
1255 Юрьевецкая
41 Иркутская область
1256 Аларская
1257 Ангарская
1258 Балаганская
1259 Баяндаевская
1260 Бодайбинская
1261 Боханская
1262 Братская городская
1263 Братская
1264 Жигаловская
1265 Заларинская
1266 Зиминская городская
1267 Зиминская
1268 Иркутская городская N 1
1269 Иркутская городская N 2
1270 Иркутская городская N 3 
1271 Иркутская городская N 4
1272 Иркутская
1273 Казачинско-Ленская
1274 Катангская
1275 Качугская
1276 Киренская
1277 Куйтунская
1278 Мамско-Чуйская
1279 Нижнеилимская
1280 Нижнеудинская
1281 Нукутская 
1282 Ольхонская
1283 Осинская
1284 Саянская
1285 Свирская
1286 Слюдянская
1287 Тайшетская
1288 Тулунская городская
1289 Тулунская
1290 Усолье-Сибирская городская
1291 Усольская
1292 Усть-Илимская городская
1293 Усть-Илимская
1294 Усть-Кутская


1641 Рошальская городская
1642 Рузская городская
1643 Сергиево-Посадская
1644 Серебряно-Прудская городская
1645 Серпуховская городская
1646 Серпуховская
1647 Солнечногорская
1648 Ступинская городская
1649 Талдомская
1650 Фрязинская городская
1651 Химкинская городская
1652 Черноголовская городская
1653 Чеховская городская
1654 Шатурская городская
1655 Шаховская городская
1656 Щелковская
1657 Электрогорская городская
1658 Электростальская городская
1659 Балашихинская городская № 2
1660 Королевская городская № 2
1661 Красногорская городская № 2
1662 Одинцовская № 2
1663 Раменская № 2
1664 Химкинская городская № 2
53 Мурманская область
1665 Апатитская
1666 Видяевская
1667 Заозерская
1668 Кандалакшская
1669 Кировская
1670 Ковдорская
1671 Кольская
1672 Ловозерская
1673 Мончегорская
1674 Мурманская
1675 Оленегорская
1676 Островнинская
1677 Печенгская
1678 Полярнозоринская
1679 Североморская
1680 ТИК ЗАТО Александровск
1681 Терская
54 Нижегородская область
1682 Ардатовская
1683 Арзамасская
168

2043 Шолоховская
63 Рязанская область
2044 Ермишинская
2045 Захаровская
2046 Кадомская
2047 Касимовская
2048 Касимовская городская
2049 Kлепиковская
2050 Кораблинская
2051 Милославская
2052 Mихайловская
2053 Александро-Невская
2054 Пителинская
2055 Пронская
2056 Путятинская
2057 Рыбновская
2058 Ряжская
2059 Рязанская
2060 Рязань, Железнодорожная
2061 Рязань, Московская
2062 Рязань, Октябрьская
2063 Рязань, Советская
2064 Сапожковская
2065 Cараевская
2066 Cacовская
2067 Сасовская городская
2068 Скопинская
2069 Скопинская городская
2070 Спасская
2071 Старожиловская
2072 Ухоловская
2073 Чучковская
2074 Шацкая
2075 Шиловская
64 Самарская область
2076 Алексеевская
2077 Безенчукская
2078 Богатовская
2079 Большеглушицкая
2080 Большечерниговская
2081 Борская
2082 Волжская
2083 Елховская
2084 Жигулевская городская
2085 Исаклинская
2086 Камышлинская
2087 Кинель-Черкасская
2088 Кинельская
2089 Кинельская городская
2090 Клявлинская
2091 Кошкинская
2092 Красноармейская
2093 Красноярская
2094 Нефтег

2441 Нижнетавдинская
2442 Омутинская
2443 Сладковская
2444 Сорокинская
2445 Тобольская
2446 Тобольская городская
2447 Тюменская
2448 Тюмень, Калининская
2449 Тюмень, Ленинская
2450 Тюмень, Центральная
2451 Уватская
2452 Упоровская
2453 Юргинская
2454 Ялуторовская
2455 Ялуторовская городская
2456 Ярковская
2457 Тюмень, Восточная
74 Ульяновская область
2458 Базарносызганская
2459 Барышская
2460 Вешкаймская
2461 Димитровградская городская
2462 Инзенская
2463 Карсунская
2464 Кузоватовская
2465 Майнская
2466 Мелекесская
2467 Николаевская
2468 Новомалыклинская
2469 Новоспасская
2470 Павловская
2471 Радищевская
2472 Сенгилеевская
2473 Старокулаткинская
2474 Старомайнская
2475 Сурская
2476 Тереньгульская
2477 Ульяновск, Железнодорожная
2478 Ульяновск, Заволжская N1
2479 Ульяновск, Заволжская N2
2480 Ульяновск, Засвияжская N1
2481 Ульяновск, Засвияжская N2
2482 Ульяновск, Ленинская
2483 Ульяновская
2484 Цильнинская
2485 Чердаклинская
2486 Новоульяновская
75 Челябинская область
2487 Агаповская
2

2762 Чукотская
84 Ямало-Ненецкий автономный округ
2763 Губкинская городская
2764 Красноселькупская
2765 Лабытнангская городская
2766 Муравленковская городская
2767 Надымская
2768 Новоуренгойская городская
2769 Ноябрьская городская
2770 Приуральская
2771 Пуровская
2772 Салехардская городская
2773 Тазовская
2774 Шурышкарская
2775 Ямальская
85 Город Байконур (Республика Казахстан)
0 tiks
86 Территория за пределами РФ
0 tiks


-1528228973.7088826

In [36]:
res = pd.concat(list_of_correct_tables, axis=1)
res.shape

(22, 97304)

### Отдельно обработаем последние 2 региона, т.к. у них нету ТИКов

In [43]:
regions_without_tiks = []

for region in regions[-2:]:
    
    try:
        region_name = region.text
        region_url = region.attrib['href']
        print(r, region_name)
    
        response = requests.get(region_url)
        assert response.status_code == 200
    except:
        print("Unexpected error:", sys.exc_info()[0], region_name)
        continue
    
    region_page_body = html.document_fromstring(response.text).getchildren().pop()
    uiks_url = region_page_body.xpath('//td//a[text()="сайт избирательной комиссии субъекта Российской Федерации"]/@href').pop()

    response = requests.get(uiks_url)
    assert response.status_code == 200

    uiks_page_body = html.document_fromstring(response.text).getchildren().pop()

    table_uiks = uiks_page_body.xpath('//td//div//table').pop()

    table = pd.read_html(html.tostring(table_uiks), header=0).pop()
    table.drop(12, inplace=True)

    region_row = [region_name for x in range(len(table.columns))]
    tik_row = [None for x in range(len(table.columns))]
    df = pd.DataFrame([region_row, tik_row], columns=table.columns)
    df = df.append(table, ignore_index=True)
    regions_without_tiks.append(df)

87 Город Байконур (Республика Казахстан)
87 Территория за пределами РФ


In [178]:
res = pd.concat(list_of_correct_tables + regions_without_tiks, axis=1)
res.shape


(22, 97704)

In [179]:
res

,УИК №1,УИК №2,УИК №3,УИК №4,УИК №5,УИК №6,УИК №7,УИК №8,УИК №9,УИК №10,...,УИК №8425,УИК №8427,УИК №8428,УИК №8429,УИК №8432,УИК №8433,УИК №8434,УИК №8436,УИК №8437,УИК №8443
0,Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),...,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ,Территория за пределами РФ
1,Адыгейская,Адыгейская,Адыгейская,Адыгейская,Адыгейская,Адыгейская,Гиагинская,Гиагинская,Гиагинская,Гиагинская,...,None,None,None,None,None,None,None,None,None,None
2,2256,2700,2858,2066,700,1044,2474,1818,2402,1772,...,1638,1668,232,1287,657,436,1141,78,848,2004
3,2181,2633,2752,2034,714,1036,2395,1770,2318,1783,...,4970,3500,1000,4931,900,2500,3000,1507,2500,4000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,36,628,241,0,0,0,0,0
5,2107,2575,2664,1857,676,954,1792,1543,2073,1640,...,1627,1635,196,654,416,431,1136,78,743,1978
6,62,41,75,142,11,68,200,111,173,126,...,5,33,0,5,0,5,5,0,105,26
7,12,17,13,35,27,14,403,116,72,17,...,3338,1832,768,3643,243,2064,1859,1429,1652,1995
8,62,41,75,142,11,68,200,111,173,126,...,5,33,36,633,241,5,5,0,105,26
9,2107,2575,2664,1857,676,954,1792,1543,2073,1640,...,1626,1635,196,654,416,431,1136,78,743,1977


In [180]:
res = res.T.reset_index()

In [181]:
res

,index,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,УИК №1,Республика Адыгея (Адыгея),Адыгейская,2256,2181,0,2107,62,12,62,...,0,0,0 0.00%,137 6.32%,32 1.48%,1977 91.15%,14 0.65%,0 0.00%,1 0.05%,5 0.23%
1,УИК №2,Республика Адыгея (Адыгея),Адыгейская,2700,2633,0,2575,41,17,41,...,0,0,15 0.57%,86 3.29%,65 2.48%,2389 91.32%,13 0.50%,5 0.19%,6 0.23%,15 0.57%
2,УИК №3,Республика Адыгея (Адыгея),Адыгейская,2858,2752,0,2664,75,13,75,...,0,0,1 0.04%,62 2.26%,13 0.47%,2645 96.57%,6 0.22%,3 0.11%,4 0.15%,0 0.00%
3,УИК №4,Республика Адыгея (Адыгея),Адыгейская,2066,2034,0,1857,142,35,142,...,0,0,5 0.25%,288 14.41%,12 0.60%,1642 82.14%,21 1.05%,6 0.30%,2 0.10%,2 0.10%
4,УИК №5,Республика Адыгея (Адыгея),Адыгейская,700,714,0,676,11,27,11,...,0,0,2 0.29%,44 6.40%,6 0.87%,624 90.83%,6 0.87%,0 0.00%,0 0.00%,2 0.29%
5,УИК №6,Республика Адыгея (Адыгея),Адыгейская,1044,1036,0,954,68,14,68,...,0,0,1 0.10%,38 3.72%,21 2.05%,952 93.15%,2 0.20%,2 0.20%,2 0.20%,1 0.10%
6,УИК №7,Республика Адыгея (Адыгея),Гиагинская,2474,2395,0,1792,200,403,200,...,0,0,9 0.45%,243 12.20%,82 4.12%,1599 80.27%,18 0.90%,11 0.55%,10 0.50%,0 0.00%
7,УИК №8,Республика Адыгея (Адыгея),Гиагинская,1818,1770,0,1543,111,116,111,...,0,0,7 0.42%,151 9.13%,69 4.17%,1388 83.92%,9 0.54%,7 0.42%,2 0.12%,5 0.30%
8,УИК №9,Республика Адыгея (Адыгея),Гиагинская,2402,2318,0,2073,173,72,173,...,0,0,9 0.40%,284 12.64%,85 3.78%,1802 80.23%,7 0.31%,17 0.76%,6 0.27%,4 0.18%
9,УИК №10,Республика Адыгея (Адыгея),Гиагинская,1772,1783,0,1640,126,17,126,...,0,0,12 0.68%,160 9.06%,69 3.91%,1484 84.03%,10 0.57%,6 0.34%,2 0.11%,3 0.17%


### Заметил, что со страниц, где таблица УИКов состоит из одной колонки, парсер стянул еще одну пустую колонку. Например выглядит так:

In [182]:
link = "http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=true&root=662000024&tvd=26620002235967&vrn=100100084849062&prver=0&pronetvd=null&region=66&sub_region=66&type=227&vibid=26620002235967"
rr = requests.get(link)

pp = html.document_fromstring(rr.text).getchildren().pop()

tb = pp.xpath('//td//div//table').pop()

tb = pd.read_html(html.tostring(tb), header=0).pop()
tb

,УИК №2604,Unnamed: 1
0,1726,NaN
1,1700,NaN
2,0,NaN
3,1286,NaN
4,11,NaN
5,403,NaN
6,11,NaN
7,1286,NaN
8,13,NaN
9,1284,NaN


### Заметил это, пытаясь избавиться от значений процентов в ячейках с кандидатами:

In [183]:
drops = []
for i, x in enumerate(res[14]):
    try:
        x = x.split()[0]
    except:
        print(x, i)
        drops.append(i)

nan 21537
nan 54273
nan 55762
nan 77207
nan 79826
nan 80053
nan 82647


In [184]:
res.loc[drops]

,index,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
21537,Unnamed: 1,Камчатский край,Алеутская,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54273,Unnamed: 1,Московская область,Восходненская поселковая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55762,Unnamed: 1,Московская область,Молодежная поселковая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77207,Unnamed: 1,Сахалинская область,Северо-Курильская,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79826,Unnamed: 1,Свердловская область,Староуткинская поселковая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80053,Unnamed: 1,Свердловская область,Уральская поселковая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82647,Unnamed: 1,Тверская область,Солнечная поселковая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
res.drop(drops, inplace=True)

In [186]:
res.shape

(97697, 23)

In [187]:
res[[2,3,4,5,6,7,8,9,10,11,12,13]].astype(int).sum()

2     109004267
3     102929412
4        219648
5      68585424
6       4821914
7      29301480
8       5039818
9      68536584
10       791226
11     72785176
12         1050
13          104
dtype: int64

In [188]:
res.loc[:, [14,15,16,17,18,19,20,21]] = res.loc[:, [14,15,16,17,18,19,20,21]].apply(lambda y: y.apply(lambda x: int(x.split()[0])))

In [189]:
res

,index,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,УИК №1,Республика Адыгея (Адыгея),Адыгейская,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
1,УИК №2,Республика Адыгея (Адыгея),Адыгейская,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
2,УИК №3,Республика Адыгея (Адыгея),Адыгейская,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
3,УИК №4,Республика Адыгея (Адыгея),Адыгейская,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
4,УИК №5,Республика Адыгея (Адыгея),Адыгейская,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2
5,УИК №6,Республика Адыгея (Адыгея),Адыгейская,1044,1036,0,954,68,14,68,...,0,0,1,38,21,952,2,2,2,1
6,УИК №7,Республика Адыгея (Адыгея),Гиагинская,2474,2395,0,1792,200,403,200,...,0,0,9,243,82,1599,18,11,10,0
7,УИК №8,Республика Адыгея (Адыгея),Гиагинская,1818,1770,0,1543,111,116,111,...,0,0,7,151,69,1388,9,7,2,5
8,УИК №9,Республика Адыгея (Адыгея),Гиагинская,2402,2318,0,2073,173,72,173,...,0,0,9,284,85,1802,7,17,6,4
9,УИК №10,Республика Адыгея (Адыгея),Гиагинская,1772,1783,0,1640,126,17,126,...,0,0,12,160,69,1484,10,6,2,3


In [190]:
res[[14,15,16,17,18,19,20,21]].astype(int).sum()

14      478994
15     8658965
16     4154880
17    56428669
18     1237969
19      499317
20      556779
21      769603
dtype: int64

In [191]:
res.columns = ['UIK', 'region', 'TIK'] + [x for x in range(1, 21)]
res

,UIK,region,TIK,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,УИК №1,Республика Адыгея (Адыгея),Адыгейская,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
1,УИК №2,Республика Адыгея (Адыгея),Адыгейская,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
2,УИК №3,Республика Адыгея (Адыгея),Адыгейская,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
3,УИК №4,Республика Адыгея (Адыгея),Адыгейская,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
4,УИК №5,Республика Адыгея (Адыгея),Адыгейская,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2
5,УИК №6,Республика Адыгея (Адыгея),Адыгейская,1044,1036,0,954,68,14,68,...,0,0,1,38,21,952,2,2,2,1
6,УИК №7,Республика Адыгея (Адыгея),Гиагинская,2474,2395,0,1792,200,403,200,...,0,0,9,243,82,1599,18,11,10,0
7,УИК №8,Республика Адыгея (Адыгея),Гиагинская,1818,1770,0,1543,111,116,111,...,0,0,7,151,69,1388,9,7,2,5
8,УИК №9,Республика Адыгея (Адыгея),Гиагинская,2402,2318,0,2073,173,72,173,...,0,0,9,284,85,1802,7,17,6,4
9,УИК №10,Республика Адыгея (Адыгея),Гиагинская,1772,1783,0,1640,126,17,126,...,0,0,12,160,69,1484,10,6,2,3


In [192]:
res.to_csv("data.csv", sep=";")